In [1]:
import streamlit as st
from textblob import TextBlob
import speech_recognition as sr
import tempfile
import os
from pydub import AudioSegment

# Task Suggestion Function
def suggest_task(emotion):
    tasks = {
        "happy": "Share your happiness with friends!",
        "sad": "Take a short walk or talk to a friend.",
        "angry": "Take deep breaths, drink water.",
        "fear": "Write down what's worrying you.",
        "neutral": "Focus on your current goals!",
        "surprise": "Pause and reflect on the new experience.",
        "disgust": "Step away from the situation for clarity."
    }
    return tasks.get(emotion.lower(), "Relax and take care of yourself.")

# Text-Based Emotion Detection
def detect_text_emotion(text):
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    if polarity > 0.3:
        return "happy"
    elif polarity < -0.3:
        return "sad"
    else:
        return "neutral"

# Audio-Based Emotion Detection (using speech to text + TextBlob)
def detect_audio_emotion(audio_file):
    recognizer = sr.Recognizer()
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp_wav:
        audio = AudioSegment.from_file(audio_file)
        audio.export(tmp_wav.name, format="wav")

    try:
        with sr.AudioFile(tmp_wav.name) as source:
            audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data)
            emotion = detect_text_emotion(text)
            return emotion, text
    except sr.UnknownValueError:
        return "Could not understand the audio.", ""
    except sr.RequestError:
        return "Error connecting to Google's service.", ""
    finally:
        os.remove(tmp_wav.name)

# Streamlit UI
st.set_page_config(page_title="Emotion Detector & Task Suggester", page_icon="🧠🎧", layout="centered")
st.title("🧠🎧 Emotion Detection & Task Recommender")

mode = st.selectbox("Choose Input Type:", ["Text", "Audio"])

if mode == "Text":
    user_text = st.text_area("Enter your thoughts:")
    if st.button("Analyze Text"):
        if user_text:
            emotion = detect_text_emotion(user_text)
            task = suggest_task(emotion)
            st.success(f"Detected Emotion: **{emotion}**")
            st.info(f"💡 Suggested Task: {task}")
        else:
            st.warning("Please enter some text.")

elif mode == "Audio":
    uploaded_file = st.file_uploader("Upload an audio file (WAV, MP3, OGG)", type=['wav', 'mp3', 'ogg'])
    if st.button("Analyze Audio"):
        if uploaded_file:
            emotion, transcript = detect_audio_emotion(uploaded_file)
            if transcript:
                task = suggest_task(emotion)
                st.success(f"Transcript: {transcript}")
                st.success(f"Detected Emotion: **{emotion}**")
                st.info(f"💡 Suggested Task: {task}")
            else:
                st.error(emotion)
        else:
            st.warning("Please upload an audio file.")


2025-04-29 17:56:18.281 
  command:

    streamlit run D:\anaconda\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-29 17:56:18.286 Session state does not function when running a script without `streamlit run`
